# Administrative Data Cleaning

This notebook shows transparently the cleaning proccess of the student data used for the project. 

The raw dataframe used was shared to us by Universidad del Pacífico and contains anonymized data on undergraduate students enrolled in the period from 2018 to 2023 in the programs of Business Administration, Finance, Accounting, Business Engineering, Information Engineering, International Business, Law, Marketing, and Economics.

Said dataset can be found <a href="https://datasets.up.edu.pe/dataset.xhtml?persistentId=hdl:20.500.14139/8RFSFQ">here</a>

## Setup

In [1]:
import pandas as pd 
from IPython.display import HTML
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

### Importing Data

In [2]:
os.getcwd()

'c:\\Users\\Matias Villalba\\Documents\\GitHub\\VRI_UP_Networks\\administrative_data\\05-CODE'

In [2]:
#importing relevant dataframes
df = pd.read_excel(r'../01-DATA_RAW\Evaluaciones de alumnos de Pregrado del 2018-2022_VERSION_5.0.xlsx', sheet_name="Dataset", header=None, usecols="A:GU", skiprows=1, nrows=272284)
df_dicc = pd.read_excel(r'../01-DATA_RAW\Evaluaciones de alumnos de Pregrado del 2018-2022_VERSION_5.0.xlsx', sheet_name="Diccionario de datos", header=0, usecols="B:F")

### Renaming vars. according to variable dictionary

In [3]:
display(df_dicc)

,Tipo de variable,Ubig,Variable,Descripción,Observaciones
0,Key,1.1,CODIGO_DEL_ALUMNO,Código anonimizado del alumno,NaN
1,Key,1.2,ACADEMIC_YEAR,Año Académico,NaN
2,Key,1.3,ACADEMIC_TERM,Periodo Académico,"I PER = Primer Semestre, II PER=Segundo Semestre"
3,Perfil,2.1,GENERO,Género del Alumno,"F=Femenino, M=Masculino"
4,Perfil,2.2,EDAD_INGRESO,Edad del alumno al ingresar a la UP,NaN
...,...,...,...,...,...
198,Hábitos de salud,14.6,HABITO_SALUD6,6. ¿Cuántas veces a la semana realiza activida...,NaN
199,Hábitos de salud,14.7,HABITO_SALUD7,"7. En el último mes, ¿has fumado tabaco?",NaN
200,Hábitos de salud,14.8,HABITO_SALUD8,"8. En el último mes, ¿has tomado una bebida qu...",NaN
201,Hábitos de salud,14.9,HABITO_SALUD9,9. ¿Alguna vez has consumido marihuana?,NaN


In [4]:
# HTML format for the title and content
html_output = """
<div style='max-height: 500px; overflow-y: auto;'>
    <h1>Diccionario de variables</h1>
"""

# Initialize a variable to track the current category
current_category = None

# Iterating through the DataFrame and formatting each row in HTML
for _, row in df_dicc.iterrows():
    # Check if the category has changed
    if row['Tipo de variable'] != current_category:
        # Update the current category
        current_category = row['Tipo de variable']
        # Add category as subtitle
        html_output += f"<h2>{current_category}</h2>"
    
    # Add variable details
    html_output += f"<p style='margin-left: 20px;'><strong>{row['Ubig']}:</strong> <strong>{row['Variable']}</strong>. {row['Descripción']}."
    if pd.notna(row['Observaciones']):
        html_output += f"<br><small>[{row['Observaciones']}]</small>"
    html_output += "</p>"

# Closing the div tag
html_output += "</div>"

# Display the HTML output with scrollbar in a Jupyter Notebook cell
display(HTML(html_output))

In [5]:
df.columns = df_dicc['Variable'].values

## Data Cleaning

### Displaying initial state of data

In [6]:
# Apply CSS to left-align headers, prevent text wrapping, and set max width to header width
display(HTML("""
<style>
    .dataframe th, .dataframe td {
        white-space: nowrap;
        max-width: 300px; 
        overflow: hidden;
        text-overflow: ellipsis;
    }
    .dataframe th, .dataframe td {
        text-align: left !important;
    }
</style>
"""))

# Now display your DataFrame
with pd.option_context(
    "display.min_rows", 20,
    'display.max_columns', None
    ):
        display(df)

,CODIGO_DEL_ALUMNO,ACADEMIC_YEAR,ACADEMIC_TERM,GENERO,EDAD_INGRESO,EDAD_ACTUAL,LUGAR_DE_NACIMIENTO,LUGAR_DE_ORIGEN,LUGAR_DE_RESIDENCIA,GRADO_PADRE,OCUPACION_PADRE,UBIGEO_PADRE,GRADO_MADRE,OCUPACION_MADRE,UBIGEO_MADRE,NUMERO_HERMANOS,NUMERO_HERMANOS_EN_LA_UP,COLEGIO_DE_PROCEDENCIA,TIPO_COLEGIO,UBIGEO_COLEGIO,BACHILLERATO_INTERNACIONAL,COLEGIO_DE_ALTO_RENDIMIENTO,EVALUACION_VIGESIMAL_COMPETENCIAS_MIXTO,3RO_MATEMATICA,3RO_COMUNICACION,4TO_MATEMATICA,4TO_COMUNICACION,5TO_MATEMATICA,5TO_COMUNICACION,3RO_CIENCIA_TECNOLOGIA_DISEÑA,3RO_CIENCIA_TECNOLOGIA_EXPLICA,3RO_CIENCIA_TECNOLOGIA_INDAGA,3RO_CIENCIAS_SOCIALES_CONSTRUYE,3RO_CIENCIAS_SOCIALES_ESPACIO_AMBIENTE,3RO_CIENCIAS_SOCIALES_RECURSOS_ECOLOGICOS,3RO_COMUNICACION_LENGUA_ESCRIBE,3RO_COMUNICACION_LENGUA_LEE,3RO_COMUNICACION_LENGUA_COMUNICA,3RO_MATEMATICA_CANTIDAD,3RO_MATEMATICA_FORMA,3RO_MATEMATICA_GESTION_DATOS,3RO_MATEMATICA_REGULARIDAD_EQUIVALENCIA,4TO_CIENCIA_TECNOLOGIA_DISEÑA,4TO_CIENCIA_TECNOLOGIA_EXPLICA,4TO_CIENCIA_TECNOLOGIA_INDAGA,4TO_CIENCIAS_SOCIALES_CONSTRUYE,4TO_CIENCIAS_SOCIALES_ESPACIO_AMBIENTE,4TO_CIENCIAS_SOCIALES_RECURSOS_ECOLOGICOS,4TO_COMUNICACION_LENGUA_ESCRIBE,4TO_COMUNICACION_LENGUA_LEE,4TO_COMUNICACION_LENGUA_COMUNICA,4TO_MATEMATICA_CANTIDAD,4TO_MATEMATICA_FORMA,4TO_MATEMATICA_GESTION_DATOS,4TO_MATEMATICA_REGULARIDAD_EQUIVALENCIA,5TO_CIENCIA_TECNOLOGIA_DISEÑA,5TO_CIENCIA_TECNOLOGIA_EXPLICA,5TO_CIENCIA_TECNOLOGIA_INDAGA,5TO_CIENCIAS_SOCIALES_CONSTRUYE,5TO_CIENCIAS_SOCIALES_ESPACIO_AMBIENTE,5TO_CIENCIAS_SOCIALES_RECURSOS_ECOLOGICOS,5TO_COMUNICACION_LENGUA_ESCRIBE,5TO_COMUNICACION_LENGUA_LEE,5TO_COMUNICACION_LENGUA_COMUNICA,5TO_MATEMATICA_CANTIDAD,5TO_MATEMATICA_FORMA,5TO_MATEMATICA_GESTION_DATOS,5TO_MATEMATICA_REGULARIDAD_EQUIVALENCIA,MODALIDAD_DE_ADMISION,AÑO_INGRESO_CARRERA,SEMESTRE_INGRESO_CARRERA,BECA,PIDIO_RECATEGORIZACION_DE_ESCALA,ESCALA_DE_PAGO_INICIAL,AÑO_DE_SOLICITUD_DE_RECATEGORIZACION,ESCALA_DE_PAGO_REGATEGORIZADO,ASIGNACION_ESCALA_INGRESOS_ESTIMADOS,ASIGNACION_ESCALA_MONTO_PAGADO_COLEGIO_PROCEDENCIA,INGRESO_ESTIMADO_FACTORIZADO_POR_NUMERO_DE_FAMILIARES,RECATEGORIZACION_INGRESOS_ESTIMADOS,RECATEGORIZACION_MONTO_PAGADO_COLEGIO_PROCEDENCIA,RECATEGORIZACION_INGRESO_ESTIMADO_FACTORIZADO_POR_NUMERO_DE_FAMILIARES,ESTADO_MATRICULA,CARRERA,PLAN_STUDIO,PRIMER_SEMESTRE_EN_QUE_CURSO_ESTUDIO,CAMBIO_DE_CARRERA,NUMERO_VECES_CAMBIO_CARRERA,CARRERA_PROCEDENTE,CARRERA_DESTINO,AGREGACION_DE_CARRERA,FACTOR_DE_INSCRIPCION,CICLO_SEGUN_REGLAMENTO,CREDITOS_ACUMULADO,CREDITOS_CICLO,CICLO_SEMESTRES_MATRICULADOS,AÑOS_ACUMULADOS_POR_ALUMNO,UBICACION_ACTUAL_DEL_ALUMNO_EN_EL_TERCIO,TERCIO_SUPERIOR,TERCIO_MEDIO,TERCIO_INFERIOR,PERIODO_DE_EGRESO_DEL_ALUMNO,TIENE_PROCESO_DISCIPLINARIO,MOTIVO_PROCESO_DISCIPLINARIO,TIENE_BAJA_ACADEMICA,MOTIVO_BAJA_ACADEMICA,RENUNCIA,CURSOS_OBLIGATORIOS_PLAN_ACADEMICO,CODIGO_DEL_CURSO,NOMBRE_DEL_CURSO,SECCION_DEL_CURSO,DOCENTE_DEL_CURSO,JEFE_DE_PRACTICA_DEL_CURSO,CREDITAJE_DEL_CURSO,ES_CURSO_NIVELATORIO,EXONERADO_CURSO_NIVELATORIO,CURSOS_EQUIVALENTES,DEPARTAMENTO_DEL_CURSO,NUMERO_DE_VECES_QUE_LLEVO_EL_CURSO,NUMERO_DE_ALUMNOS_POR_CURSO,NUMERO_DE_HORAS_TOTALES_DEL_CURSO,PORCENTAJE_DE_APROBADOS_POR_CURSO,NUMERO_DE_RETIRADOS_EN_EL_CURSO,NUMERO_DE_RETIRADOS_DE_CICLO,CALIFICACION_DEL_PROFESOR_DE_LA_SECCION,EXAMEN_FINAL_NOTA,EXAMEN_PARCIAL_NOTA,TRABAJO_NOTA,NOTA_FINAL_DEL_CURSO,PORCENTAJE_EXAMEN_PARCIAL,PORCENTAJE_EXAMEN_FINAL,PORCENTAJE_TRABAJO,PORCENTAJE_TRABAJO_FINAL,PORCENTAJE_PROMEDIO_PRACTICA,PORCENTAJE_NOTA_PARCIAL,PORCENTAJE_PRE_FINAL,PORCENTAJE_NOTA_FINAL,PORCENTAJE_PARTICIPACION,PORCENTAJE_ASISTENCIA,PORCENTAJE_CASOS,PORCENTAJE_CONTROL,PROMEDIO_ACUMULADO,PROMEDIO_CICLO,HA_TENIDO_EL_ROL_DE_JP,HA_TENIDO_EL_ROL_DE_AI_EN_EL_CIUP,NUMERO_DE_PRACTICAS_PREPROFESIONALES,N_NEUROTICISMO,N1_ANSIEDAD,N2_HOSTILIDAD,N3_DEPRESIÓN,N4_ANSIEDAD_SOCIAL,N5_IMPULSIVIDAD,N6_VULNERABILIDAD,E_EXTROVERSIÓN,E1_CORDIALIDAD,E2_GREGARISMO,E3_ASERTIVIDAD,E4_ACTIVIDAD,E5_BÚSQUEDA_DE_EMOCIONES,E6_EMOCIONES_POSITIVAS,O_APERTURA,O1_FANTASÍA,O2_ESTÉTICA,O3_SENTIMIENTOS,O4_ACCIONE

### Missing Values and Categoricals

In [7]:
df['SECCION_DEL_CURSO'] = df['SECCION_DEL_CURSO'].str.upper()

df['ACADEMIC_YEAR'] = pd.to_numeric(df['ACADEMIC_YEAR'],  errors='coerce').astype('Int64')

# Create 'ciclo' column based on conditions
df['CICLO'] = None
df.loc[df['ACADEMIC_TERM'] == "CUR VERANO", 'CICLO'] = df['ACADEMIC_YEAR'].astype(str) + "-0"
df.loc[df['ACADEMIC_TERM'] == "I PER", 'CICLO'] = df['ACADEMIC_YEAR'].astype(str) + "-1"
df.loc[df['ACADEMIC_TERM'] == "II PER", 'CICLO'] = df['ACADEMIC_YEAR'].astype(str) + "-2"

df['AÑO_INGRESO_CARRERA'] = pd.to_numeric(df['AÑO_INGRESO_CARRERA'], errors='coerce').astype('Int64')

# Create 'ciclo_ingreso' column based on conditions
df['CICLO_INGRESO_CARRERA'] = None
df.loc[df['SEMESTRE_INGRESO_CARRERA'] == "CUR VERANO", 'CICLO_INGRESO_CARRERA'] = df['AÑO_INGRESO_CARRERA'].astype(str) + "-0"
df.loc[df['SEMESTRE_INGRESO_CARRERA'] == "I PER", 'CICLO_INGRESO_CARRERA'] = df['AÑO_INGRESO_CARRERA'].astype(str) + "-1"
df.loc[df['SEMESTRE_INGRESO_CARRERA'] == "II PER", 'CICLO_INGRESO_CARRERA'] = df['AÑO_INGRESO_CARRERA'].astype(str) + "-2"

In [8]:
df[['mat_primer_ciclo_en_que_estudio1', 'mat_primer_ciclo_en_que_estudio2']] = df['PRIMER_SEMESTRE_EN_QUE_CURSO_ESTUDIO'].str.split("-", expand=True)

df['mat_primer_ciclo_en_que_estudio1'] = pd.to_numeric(df['mat_primer_ciclo_en_que_estudio1'], errors='coerce').astype('Int64')

#Create 'primer_ciclo' column based on conditions
df['PRIMER_CICLO'] = None
df.loc[df['mat_primer_ciclo_en_que_estudio2'] == "CUR VERANO", 'PRIMER_CICLO'] = df['mat_primer_ciclo_en_que_estudio1'].astype(str) + "-0"
df.loc[df['mat_primer_ciclo_en_que_estudio2'].isin(["I PER", "01"]), 'PRIMER_CICLO'] = df['mat_primer_ciclo_en_que_estudio1'].astype(str) + "-1"
df.loc[df['mat_primer_ciclo_en_que_estudio2'] == "II PER", 'PRIMER_CICLO'] = df['mat_primer_ciclo_en_que_estudio1'].astype(str) + "-2"

# Step 4: Drop intermediate columns
df = df.drop(columns=['mat_primer_ciclo_en_que_estudio1', 'mat_primer_ciclo_en_que_estudio2'])

In [9]:
columns = [
    'GRADO_MADRE', 
    'GRADO_PADRE', 
    'OCUPACION_MADRE', 
    'OCUPACION_PADRE', 
    'UBIGEO_MADRE', 
    'UBIGEO_PADRE', 
    'MODALIDAD_DE_ADMISION'
]

# Step 1: Replace "FALTANTE" with an empty string ("") in the specified columns
for var in columns:
    df[var] = df[var].replace("FALTANTE", np.nan)

In [10]:
# Replace "OPCIONAL" with an empty string in the column cole_tipo_colegio
df['TIPO_COLEGIO'] = df['TIPO_COLEGIO'].replace("OPCIONAL", np.nan)

# Replace "COAR" with "Colegios de Alto Rendimiento" in the column cole_tipo_colegio
df['TIPO_COLEGIO'] = df['TIPO_COLEGIO'].replace("COAR", "Colegios de Alto Rendimiento")

# Replace "OPCIONAL" with an empty string in the column cole_eva_viges_competen_mixto
df['EVALUACION_VIGESIMAL_COMPETENCIAS_MIXTO'] = df['EVALUACION_VIGESIMAL_COMPETENCIAS_MIXTO'].replace("OPCIONAL", np.nan)

# Replace "SIN BECA" with an empty string in the column pen_beca
df['BECA'] = df['BECA'].replace("SIN BECA", np.nan)

# Replace 0 with NaN (equivalent to Stata's `.`) in the column pen_ano_de_solicit_de_recatego
df['AÑO_DE_SOLICITUD_DE_RECATEGORIZACION'] = df['AÑO_DE_SOLICITUD_DE_RECATEGORIZACION'].replace(0, np.nan)

In [11]:
df

,CODIGO_DEL_ALUMNO,ACADEMIC_YEAR,ACADEMIC_TERM,GENERO,EDAD_INGRESO,EDAD_ACTUAL,LUGAR_DE_NACIMIENTO,LUGAR_DE_ORIGEN,LUGAR_DE_RESIDENCIA,GRADO_PADRE,...,HABITO_SALUD4,HABITO_SALUD5,HABITO_SALUD6,HABITO_SALUD7,HABITO_SALUD8,HABITO_SALUD9,HABITO_SALUD10,CICLO,CICLO_INGRESO_CARRERA,PRIMER_CICLO
0,0x003c4717dc61e2a801539565cd9e8167ad6645fa175a...,2018,CUR VERANO,M,19,29,TACNA-JORGE BASADRE-ILABAYA,Provincia,LIMA-LIMA-SAN ISIDRO,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-0,2013-1,2013-1
1,0x003c4717dc61e2a801539565cd9e8167ad6645fa175a...,2018,CUR VERANO,M,19,29,TACNA-JORGE BASADRE-ILABAYA,Provincia,LIMA-LIMA-SAN ISIDRO,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-0,2013-1,2013-1
2,0x0090bcfbb9f54edfce3a743cbed4b2b540a7afd08269...,2018,CUR VERANO,F,16,25,LIMA-LIMA-MIRAFLORES,Lima,LIMA-LIMA-MIRAFLORES,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-0,2015-1,2015-1
3,0x00c5536b4ceda92accead7b9258e738be746b2dc0d98...,2018,CUR VERANO,F,17,24,LIMA-LIMA-RIMAC,Lima,LIMA-LIMA-RÍMAC,Titulo Profesional,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-0,2017-1,2017-1
4,0x00d135a1b90acfdb8567ee23a352ee17f36a8c7c912e...,2018,CUR VERANO,M,17,25,CARACAS--,Exterior,LIMA-LIMA-PUEBLO LIBRE,Titulo Profesional,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-0,2016-1,2016-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272279,0xfff51cd8d8ecd339d976bbd679fe2069542342e02ec7...,2022,II PER,M,18,20,CALLAO-CALLAO-CALLAO,Lima,CALLAO-CALLAO-CALLAO,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-2,2021-2,2021-2
272280,0xfff51cd8d8ecd339d976bbd679fe2069542342e02ec7...,2022,II PER,M,18,20,CALLAO-CALLAO-CALLAO,Lima,CALLAO-CALLAO-CALLAO,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-2,2021-2,2021-2
272281,0xfff51cd8d8ecd339d976bbd679fe2069542342e02ec7...,2022,II PER,M,18,20,CALLAO-CALLAO-CALLAO,Lima,CALLAO-CALLAO-CALLAO,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-2,2021-2,2021-2
272282,0xfff51cd8d8ecd339d976bbd679fe2069542342e02ec7...,2022,II PER,M,18,20,CALLAO-CALLAO-CALLAO,Lima,CALLAO-CALLAO-CALLAO,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-2,2021-2,2021-2


### Duplicates

In [12]:
def duplicates_report(df, subset):
    total_duplicates = df.duplicated(subset=subset).sum()

    duplicates = df[df.duplicated(subset=subset, keep=False)].copy()
    if subset == None:
        duplicates['combined'] = duplicates.apply(tuple, axis=1)
    else:    
        duplicates['combined'] = duplicates[subset].apply(tuple, axis=1)
    duplicate_report = duplicates.groupby('combined').size().reset_index(name='count')
    duplicate_report = duplicate_report['count'].value_counts().sort_index()

    html = (f"<h2>Duplicates Report</h2>")
    
    if total_duplicates > 0:
        html += (f"<table border='1' class='dataframe'>"
                f"<tr><th><b>Copies</b></th><th><b>Unique obs.</b></th><th><b>Total obs.</b></th><th><b>Surplus</b></th></tr>")

        for x, y in duplicate_report.items():
            html += f"<tr><td>{x} observations: </td><td>{y}</td><td>{y*x}</td><td>{y*x-y}</td></tr>"

        # End HTML table
        html += (f"</table><br>")
    html += (f"Total number of duplicates: <b>{total_duplicates}</b>")

    display(HTML(html))

In [13]:
duplicates_report(df, None)

Copies,Unique obs.,Total obs.,Surplus
2 observations:,3043,6086,3043
3 observations:,36,108,72
4 observations:,154,616,462


In [14]:
df = df.drop_duplicates().reset_index(drop=True)

### IDs

IDs should be the **student code**, the **academic term**, the **course code**, and the **course section**. However we can see that there are some instances of duplicates. This means that for the same student, in the same academic year, course, and section, there are two different values for some other column registered.

In [15]:
ids = ['CODIGO_DEL_ALUMNO', 'CICLO', 'CODIGO_DEL_CURSO','SECCION_DEL_CURSO']
duplicates_report(df, ids)

Copies,Unique obs.,Total obs.,Surplus
2 observations:,142,284,142


In [16]:
df[df.duplicated(subset=ids, keep=False)]

,CODIGO_DEL_ALUMNO,ACADEMIC_YEAR,ACADEMIC_TERM,GENERO,EDAD_INGRESO,EDAD_ACTUAL,LUGAR_DE_NACIMIENTO,LUGAR_DE_ORIGEN,LUGAR_DE_RESIDENCIA,GRADO_PADRE,...,HABITO_SALUD4,HABITO_SALUD5,HABITO_SALUD6,HABITO_SALUD7,HABITO_SALUD8,HABITO_SALUD9,HABITO_SALUD10,CICLO,CICLO_INGRESO_CARRERA,PRIMER_CICLO
14454,0x760754ba631b7f1d9fd2b06b7b22c581500bde25bb6b...,2018,I PER,F,16,25,LIMA-LIMA-SAN MIGUEL,Lima,LIMA-LIMA-SAN MIGUEL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-1,2015-1,2015-1
14455,0x760754ba631b7f1d9fd2b06b7b22c581500bde25bb6b...,2018,I PER,F,16,25,LIMA-LIMA-SAN MIGUEL,Lima,LIMA-LIMA-SAN MIGUEL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-1,2015-1,2015-1
64996,0x80cb52f6cd1cd0fc4f9b094146e5b28efb6f6768f5f1...,2019,I PER,M,16,26,LIMA-LIMA-JESUS MARIA,Lima,LIMA-LIMA-JESÚS MARÍA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-1,2013-1,2013-1
64997,0x80cb52f6cd1cd0fc4f9b094146e5b28efb6f6768f5f1...,2019,I PER,M,16,26,LIMA-LIMA-JESUS MARIA,Lima,LIMA-LIMA-JESÚS MARÍA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-1,2013-1,2013-1
71213,0xc0db24df9a9f9a67ef517a2a3341d1538ea4c6996cd4...,2019,I PER,M,18,26,LIMA-LIMA-ANCON,Lima,LIMA-LIMA-LIMA,Grado Acádemico de Bachiller,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-1,2016-1,2018-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209067,0xf1c8dd53e4f076990eaca9c0e9547902f54b67fd64e4...,2021,II PER,M,17,22,LIMA-LIMA-LINCE,Lima,LIMA-LIMA-LIMA,Técnico Superior,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-2,2018-2,2018-2
209068,0xf1c8dd53e4f076990eaca9c0e9547902f54b67fd64e4...,2021,II PER,M,17,22,LIMA-LIMA-LINCE,Lima,LIMA-LIMA-LIMA,Técnico Superior,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-2,2018-2,2018-2
209069,0xf1c8dd53e4f076990eaca9c0e9547902f54b67fd64e4...,2021,II PER,M,17,22,LIMA-LIMA-LINCE,Lima,LIMA-LIMA-LIMA,Técnico Superior,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-2,2018-2,2018-2
209070,0xf1c8dd53e4f076990eaca9c0e9547902f54b67fd64e4...,2021,II PER,M,17,22,LIMA-LIMA-LINCE,Lima,LIMA-LIMA-LIMA,Técnico Superior,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-2,2018-2,2018-2


Checking for systematic differences in values mannualy would be difficult and inneficient, because of that, we write a code that neatly shows the differing values within each duplicated unique observation

In [17]:
# Create a function to find differing values
def find_differences(dup_df, subset_cols):
    # Group by the subset columns to compare duplicates within the same group
    grouped = dup_df.groupby(subset_cols)
    
    differing_rows = []
    
    # Compare each group of duplicates
    for _, group in grouped:
        if len(group) > 1:  # Check only if there's more than one duplicate
            # Find the differences between rows
            for col in group.columns:
                if col not in subset_cols:  # Skip subset columns used to check duplicates
                    # If values in this column are not the same, mark them
                    if len(group[col].unique()) > 1:
                        differing_rows.append({
                            'Subset_Columns': str(group[subset_cols].iloc[0].to_dict()),
                            'Differing_Column': col,
                            'Differing_Values': group[col].tolist()
                        })
    
    differing_report = pd.DataFrame(differing_rows)

    wide_report = differing_report.pivot(index='Subset_Columns', columns='Differing_Column', values='Differing_Values')
    
    return wide_report

Here, each row corresponds to a unique ID, and the columns show the differing values that result in the initial duplicates drop not dropping said ID duplicates.

In [18]:
duplicates = df[df.duplicated(subset=ids, keep=False)].copy()
id_differences = find_differences(duplicates, ids)

with pd.option_context(
    "display.min_rows", 10,
    'display.max_columns', None
    ):
    display(id_differences)

Differing_Column,A1_CONFIANZA,A2_FRANQUEZA,A3_ALTRUISMO,A4_ACTITUD_CONCILIADORA,A5_MODESTIA,A6_SENSIBILIDAD_A_LOS_DEMÁS,APRENDIZAJE_ENTRE_IGUALES,AUTO_REGULACIÓN_METACOGNITIVA,A_AMABILIDAD,BECA,BÚSQUEDA_DE_AYUDA,C1_COMPETENCIA,C2_ORDEN,C3_SENTIDO_DEL_DEBER,C4_NECESIDAD_DE_LOGRO,C5_AUTODISCIPLINA,C6_DELIBERACIÓN,CALIFICACION_DEL_PROFESOR_DE_LA_SECCION,C_RESONSABILIDAD,E1_CORDIALIDAD,E2_GREGARISMO,E3_ASERTIVIDAD,E4_ACTIVIDAD,E5_BÚSQUEDA_DE_EMOCIONES,E6_EMOCIONES_POSITIVAS,ENSAYO,E_EXTROVERSIÓN,Estrategias de aprendizaje: ELABORACIÓN,Estrategias de aprendizaje: ORGANIZACIÓN,HABITO_SALUD6,N1_ANSIEDAD,N2_HOSTILIDAD,N3_DEPRESIÓN,N4_ANSIEDAD_SOCIAL,N5_IMPULSIVIDAD,N6_VULNERABILIDAD,N_NEUROTICISMO,O1_FANTASÍA,O2_ESTÉTICA,O3_SENTIMIENTOS,O4_ACCIONES,O5_IDEAS,O6_VALORES,O_APERTURA,PENSAMIENTO_CRÍTICO,REGULACIÓN_DEL_ESFUERZO,TIEMPO_Y_AMBIENTE_DE_ESTUDIO
Subset_Columns,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"{'CODIGO_DEL_ALUMNO': '0x010eeef0ea3a70753fa661fbb14bbaa563b06ef0c59d5f6b9737ac1b5bf4e4f6', 'CICLO': '2021-1', 'CODIGO_DEL_CURSO': '120133', 'SECCION_DEL_CURSO': 'D'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[6.6078, 6.0733]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"{'CODIGO_DEL_ALUMNO': '0x010eeef0ea3a70753fa661fbb14bbaa563b06ef0c59d5f6b9737ac1b5bf4e4f6', 'CICLO': '2021-1', 'CODIGO_DEL_CURSO': '1F0120', 'SECCION_DEL_CURSO': 'C'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[5.2411, 6.1786]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"{'CODIGO_DEL_ALUMNO': '0x097b73181755d5227b81c43ec5bf6a1e8161ebd84a3e2eb88886c77c0bc023c0', 'CICLO': '2021-1', 'CODIGO_DEL_CURSO': '120000', 'SECCION_DEL_CURSO': 'REC'}","[13.0, 17.0]","[17.0, 23.0]",NaN,"[21.0, 22.0]","[20.0, 21.0]","[26.0, 22.0]",NaN,NaN,"[125.0, 133.0]",NaN,NaN,"[29.0, 27.0]","[26.0, 21.0]","[26.0, 27.0]","[27.0, 25.0]","[27.0, 24.0]","[23.0, 26.0]",NaN,"[158.0, 150.0]","[28.0, 29.0]",NaN,"[27.0, 24.0]","[22.0, 21.0]","[26.0, 22.0]",NaN,NaN,"[156.0, 149.0]",NaN,NaN,NaN,"[26.0, 16.0]","[12.0, 8.0]","[19.0, 11.0]","[10.0, 11.0]","[8.0, 9.0]","[10.0, 9.0]","[85.0, 64.0]","[19.0, 12.0]","[21.0, 23.0]","[22.0, 20.0]","[12.0, 19.0]","[28.0, 24.0]","[24.0, 21.0]","[126.0, 119.0]",NaN,NaN,NaN
"{'CODIGO_DEL_ALUMNO': '0x097b73181755d5227b81c43ec5bf6a1e8161ebd84a3e2eb88886c77c0bc023c0', 'CICLO': '2021-1', 'CODIGO_DEL_CURSO': '170131', 'SECCION_DEL_CURSO': 'REC'}","[13.0, 17.0]","[17.0, 23.0]",NaN,"[21.0, 22.0]","[20.0, 21.0]","[26.0, 22.0]",NaN,NaN,"[125.0, 133.0]",NaN,NaN,"[29.0, 27.0]","[26.0, 21.0]","[26.0, 27.0]","[27.0, 25.0]","[27.0, 24.0]","[23.0, 26.0]",NaN,"[158.0, 150.0]","[28.0, 29.0]",NaN,"[27.0, 24.0]","[22.0, 21.0]","[26.0, 22.0]",NaN,NaN,"[156.0, 149.0]",NaN,NaN,NaN,"[26.0, 16.0]","[12.0, 8.0]","[19.0, 11.0]","[10.0, 11.0]","[8.0, 9.0]","[10.0, 9.0]","[85.0, 64.0]","[19.0, 12.0]","[21.0, 23.0]","[22.0, 20.0]","[12.0, 19.0]","[28.0, 24.0]","[24.0, 21.0]","[126.0, 119.0]",NaN,NaN,NaN
"{'CODIGO_DEL_ALUMNO': '0x0ef911215bce81895e16afedf12caf069bf52cc94607d92a30beef2d91937e3e', 'CICLO': '2021-1', 'CODIGO_DEL_CURSO': '138649', 'SECCION_DEL_CURSO': 'A'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[5.9286, 4.9072]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"{'CODIGO_DEL_ALUMNO': '0xf1c8dd53e4f076990eaca9c0e9547902f54b67fd64e437e36a4d82006896f406', 'CICLO': '2021-2', 'CODIGO_DEL_CURSO': '144741', 'SECCION_DEL_CURSO': 'A'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Subsidio tipo 1, Beca Patronato UP]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

We can clearly see that there is some cleaning to be done in some variable modules. 

First, there seems to be multiple values for variables of the Big Five (Características de personalidad) module. As if the same student, in the same academic term, and in the same course and section had been surveyed twice and therefore multiple responses had been registered.
The same thing happens within the Studying Strategies (Estrategias de aprendizaje) and the Health Habits (Hábitos de salud) modules.

Second, there are differing values in the rating the student gave the professor for their course section. As if the student had rated the professor twice during the same academic term.

Third, a same student has multiple scholarship categories assigned in the same term.

In [19]:
df['CALIFICACION_DEL_PROFESOR_DE_LA_SECCION'] = df.groupby(ids)['CALIFICACION_DEL_PROFESOR_DE_LA_SECCION'].transform('max')

In [20]:
df_wide = df.copy()
df_wide['pen_n_beca'] = "BECA" + (df_wide.groupby(ids).cumcount() + 1).astype(str)
df_wide = df_wide.pivot(index=ids, columns='pen_n_beca', values='BECA').reset_index()
df = df.merge(df_wide, on=ids, how='left')
df = df.drop(columns=['BECA'])

In [21]:
cols = list(df.filter(regex='^([ACENO](\\d|)_)').columns) + ['APRENDIZAJE_ENTRE_IGUALES',
    'AUTO_REGULACIÓN_METACOGNITIVA', 
    'BÚSQUEDA_DE_AYUDA', 
    'ENSAYO', 
    'Estrategias de aprendizaje: ELABORACIÓN', 
    'Estrategias de aprendizaje: ORGANIZACIÓN', 
    'PENSAMIENTO_CRÍTICO', 'REGULACIÓN_DEL_ESFUERZO', 
    'TIEMPO_Y_AMBIENTE_DE_ESTUDIO']

for col in cols:
    df[col] = df.groupby(ids)[col].transform('mean')

In [22]:
# Step 1: Define a mapping from the categorical strings to numerical values
habitos_mapping = {
    'Nunca': 0,
    'De 1 a 2 veces': 1,
    'De 3 a 4 veces': 2,
    'De 5 a 7 veces': 3
}

# Step 2: Apply the mapping to the 'HABITOS_SALUD6' column, ignoring NaN
df['HABITO_SALUD6_num'] = df['HABITO_SALUD6'].map(habitos_mapping)

# Step 3: Apply the groupby and mode (most frequent value) transformation for 'HABITOS_SALUD6'
df['HABITO_SALUD6_num'] = df.groupby(ids)['HABITO_SALUD6_num'].transform('max')

# Step 4: Map the numerical values back to the original strings
reverse_habitos_mapping = {v: k for k, v in habitos_mapping.items()}
df['HABITO_SALUD6'] = df['HABITO_SALUD6_num'].map(reverse_habitos_mapping)

# Step 5: Drop the temporary numerical column
df = df.drop(columns=['HABITO_SALUD6_num'])

In [23]:
df = df.drop_duplicates().reset_index(drop=True)
duplicates_report(df, ids)

In [24]:
for col in ['AÑO_DE_SOLICITUD_DE_RECATEGORIZACION', 'ESCALA_DE_PAGO_REGATEGORIZADO', 'AÑOS_ACUMULADOS_POR_ALUMNO', 'UBICACION_ACTUAL_DEL_ALUMNO_EN_EL_TERCIO', 'PERIODO_DE_EGRESO_DEL_ALUMNO']:   
    df[col] = df[col].astype('Int64')

## EDA

In [25]:
def col_analysis(df, col):
    if not os.path.exists(f"../../_static\\temp_figures"):
        os.makedirs(f"../../_static\\temp_figures")
        
    total_obs = len(df[col])

    unique_values = df[col].nunique()
    unique_percentage = (unique_values / total_obs) * 100
    missing_values = df[col].isna().sum()
    missing_percentage = (missing_values / total_obs) * 100

    text_info = (f"<h2>{col}</h2>"
                    f"<b>Total number of unique values:</b> {unique_values} ({unique_percentage:.2f}%)<br>"
                    f"<b>Missing values:</b> {missing_values} ({missing_percentage:.2f}%)<br>")
    
    if unique_values > 11 and unique_values < 2600:
        top_values = df[col].value_counts().head(5)
        bot_values = df[col].value_counts().tail(5)

        text_info += (f"<b>Most frequent values:</b><br>"
                        f"<div style='padding-left: 20px;'>")
        for value, count in top_values.items():
            percentage = (count / total_obs) * 100
            text_info += f"{value}: {count} ({percentage:.1f}%)<br>"
        text_info += "</div>"
        text_info += (f"<b>Least frequent values:</b><br>"
                        f"<div style='padding-left: 20px;'>")
        for value, count in bot_values.items():
            percentage = (count / total_obs) * 100
            text_info += f"{value}: {count} ({percentage:.1f}%)<br>"
        text_info += "</div>"
    elif unique_values <= 11:
        text_info += (f"<b>Value Counts:</b><br>"
                        f"<div style='padding-left: 20px;'>")
        for value, count in df[col].value_counts().items():
            percentage = (count / total_obs) * 100
            text_info += f"{value}: {count} ({percentage:.1f}%)<br>"
        text_info += "</div>"
    else:
        text_info += "<br>"
    if pd.api.types.is_numeric_dtype(df[col]):
        min_value = df[col].min()
        max_value = df[col].max()
        median_value = df[col].median()
        mode_value = df[col].mode().values
        mode_value_str = ', '.join(map(str, mode_value))
        
        text_info += (f"<b>Minimum value:</b> {min_value}<br>"
                      f"<b>Maximum value:</b> {max_value}<br>"
                      f"<b>Median value:</b> {median_value}<br>"
                      f"<b>Mode value(s):</b> {mode_value_str}<br>")

    
    # Start an HTML block for the layout
    html = f"""
    <div style="display: flex; align-items: flex-start;">
        <div style="width: 40%; padding-right: 20px; word-wrap: break-word; max-width: 95%;">
            {text_info}
        </div>
        <div style="width: 60%;">
    """

    # Plot based on the column type
    text_color = '#D3D3D3'
    plot_generated = False
    box_plot = False

    if pd.api.types.is_numeric_dtype(df[col]):
        if pd.api.types.is_integer_dtype(df[col]) and (df[col].nunique() < 12):
            # Plot a bar plot for integer data
            plot_generated = True
            fig, axs = plt.subplots(figsize=(8, 5))
            value_counts = df[col].value_counts().sort_index()
            sns.barplot(x=value_counts.index, y=value_counts.values)
            plt.title(f"Bar Plot of {col}", color=text_color)
            plt.xlabel(col, color=text_color)
            plt.ylabel("Frequency", color=text_color)
        elif pd.api.types.is_integer_dtype(df[col]):
            # Plot a bar plot for integer data
            plot_generated = True
            fig, axs = plt.subplots(nrows=2, figsize=(8, 5), sharex=True, gridspec_kw={'height_ratios': [3, 1]})
            box_plot = True
            sns.histplot(df[col].dropna(), discrete=True, ax=axs[0])
            axs[0].set_title(f"Histogram of {col}", color=text_color)
            axs[0].set_ylabel("Frequency", color=text_color)

            sns.boxplot(x=df[col].dropna(), ax=axs[1], orient='h')
            axs[1].set_xlabel(col, color=text_color)
            axs[1].set_ylabel("")
        else:
            # Plot a KDE plot for float data
            plot_generated = True
            fig, axs = plt.subplots(nrows=2, figsize=(8, 5), sharex=True, gridspec_kw={'height_ratios': [3, 1]})
            box_plot = True
            sns.histplot(df[col].dropna(), bins=20, ax=axs[0])
            axs[0].set_title(f"Histogram of {col}", color=text_color)
            axs[0].set_ylabel("Density", color=text_color)

            sns.boxplot(x=df[col].dropna(), ax=axs[1], orient='h')
            axs[1].set_xlabel(col, color=text_color)
            axs[1].set_ylabel("")
            
    if pd.api.types.is_object_dtype(df[col]) and df[col].nunique() < 10 and df[col].dropna().nunique() > 1:
        plot_generated = True
        fig, axs = plt.subplots(figsize=(8, 5))
        sorted_values = (df[col].dropna().value_counts() / df[col].dropna().value_counts().sum()*100).sort_values()

        others_sum = 0
        new_value_counts = {}
        others_added = False

        # Loop through categories and decide if they should go into "Others"
        for value, pct in sorted_values.items():
            if pct + others_sum < 5:
                others_sum += pct
                others_added = True
            else:
                new_value_counts[value] = df[col].dropna().value_counts()[value]

        if others_added:
            new_value_counts['Others'] = int((others_sum / 100) * df[col].dropna().value_counts().sum())

        # Convert the dictionary back to a series for plotting
        new_value_counts = pd.Series(new_value_counts).sort_values(ascending=False)

        # Plot pie chart
        new_value_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90, colors=plt.cm.Paired.colors)
        plt.title(f"Pie Chart of {col}", color=text_color)
        plt.ylabel('')  # Hide the y-label
        for text in axs.texts:
            if '%' not in text.get_text():
                text.set_color(text_color)

    if plot_generated and not box_plot:
        fig.patch.set_alpha(0)
        axs.patch.set_alpha(0) 
        sns.despine(left=True, bottom=True)

        axs.tick_params(axis='x', colors=text_color)
        axs.tick_params(axis='y', colors=text_color)

        for spine in axs.spines.values():
            spine.set_visible(False)

        legend = axs.get_legend()
        if legend:
            plt.setp(legend.get_texts(), color=text_color)
        
        # Save the plot as an image and embed it in HTML
        plt.tight_layout()
        plt.savefig(f"../../_static\\temp_figures\\{col}.png", transparent=True)

        # Embed the plot in the HTML
        html += f'<img src="../../_static\\temp_figures\\{col}.png" width="100%">'

        plt.close()
    
    elif plot_generated and box_plot:
        fig.patch.set_alpha(0)
        for ax in axs:
            ax.patch.set_alpha(0) 
            sns.despine(left=True, bottom=True)

            ax.tick_params(axis='x', colors=text_color)
            ax.tick_params(axis='y', colors=text_color)

            for spine in ax.spines.values():
                spine.set_visible(False)

            legend = ax.get_legend()
            if legend:
                plt.setp(legend.get_texts(), color=text_color)
            
        # Save the plot as an image and embed it in HTML
        plt.tight_layout()
        plt.savefig(f"../../_static\\temp_figures\\{col}_boxplot.png", transparent=True)

        # Embed the plot in the HTML
        html += f'<img src="../../_static\\temp_figures\\{col}_boxplot.png" width="100%">'

        plt.close()
    
    html += "</div></div><br><br>"

    # Display the HTML content
    display(HTML(html))

In [26]:
for col in df.columns:
    col_analysis(df,col)

In [27]:
from ydata_profiling import ProfileReport
import numpy as np
import pandas as pd

profile = ProfileReport(df, title="Profiling Report")